# Women In Data Science - Kaggle Competition 


## User ID: RinzlerTron ; Team: Songpyeon
## Rank - #27 on Public Leaderboard and Rank #30 on Private Leaderboard

https://www.kaggle.com/c/wids2018datathon

In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')
#from fastai.imports import *
#from fastai.structured import *
from pandas_summary import DataFrameSummary
%matplotlib inline 
import missingno as mn
from sklearn.linear_model import LogisticRegression
from catboost import *
from fancyimpute import  KNN, NuclearNormMinimization, SoftImpute
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

Using TensorFlow backend.
E:\Other\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [90]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
del train['train_id']
id = test['test_id']
del test['test_id']
train.shape, test.shape

((18255, 1234), (27285, 1233))

In [91]:
col = train.isnull().sum().sort_values(ascending = False)<0.99*train.shape[0]
train = train.loc[:,col]
train.head(3)

,AA3,AA4,AA5,AA6,AA7,AA14,AA15,DG1,is_female,DG3,...,LN2_2,LN2_3,LN2_4,LN2_RIndLngBEOth,LN2_WIndLngBEOth,GN1,GN2,GN3,GN4,GN5
0,3,32,3.0,NaN,323011,3854,481,1975,1,3,...,1,1,1,NaN,NaN,99.0,99,99,99,99
1,2,26,NaN,8.0,268131,2441,344,1981,1,8,...,1,3,4,Bengali,Bengali,NaN,1,2,2,2
2,1,16,NaN,7.0,167581,754,143,1995,1,3,...,1,2,2,Hindi,Hindi,1.0,2,2,2,2


In [92]:
index = col[(col == True)].index
test = test.loc[:,index]
del test['is_female']
test.shape, train.shape

((27285, 699), (18255, 700))

### Convert all float into int

In [93]:
col2 = list(train.select_dtypes(include=['object']).columns)
col2

['DL1_OTHERS',
 'DL2_96_OTHERS',
 'DL4_OTHERS',
 'MM12_REC',
 'FL9A_OTHERS',
 'FB19_6_OTHERS',
 'LN2_RIndLngBEOth',
 'LN2_WIndLngBEOth']

In [94]:
train.loc[:,col2] = train.loc[:,col2].fillna("missing")
test.loc[:,col2] = test.loc[:,col2].fillna("missing")

In [95]:
train["row_nulls"] = train.isnull().sum(axis=1)/float(train.shape[1])
test["row_nulls"] = test.isnull().sum(axis=1)/float(test.shape[1])
train.fillna(1000,inplace=True)
test.fillna(1000,inplace=True)

In [96]:
float_col = list(train.select_dtypes(include=['float64']).columns)

In [97]:
train[float_col] = train[float_col].astype("int64")
test[float_col] = test[float_col].astype("int64")

In [98]:
train.head(3)

,AA3,AA4,AA5,AA6,AA7,AA14,AA15,DG1,is_female,DG3,...,LN2_3,LN2_4,LN2_RIndLngBEOth,LN2_WIndLngBEOth,GN1,GN2,GN3,GN4,GN5,row_nulls
0,3,32,3,1000,323011,3854,481,1975,1,3,...,1,1,missing,missing,99,99,99,99,99,0
1,2,26,1000,8,268131,2441,344,1981,1,8,...,3,4,Bengali,Bengali,1000,1,2,2,2,0
2,1,16,1000,7,167581,754,143,1995,1,3,...,2,2,Hindi,Hindi,1,2,2,2,2,0


In [99]:
col = list(train.select_dtypes(include=['object','int64']).columns)
v = (train[col].apply(lambda x: x.nunique(), axis=0)>3).reset_index()
col_filtered = list(v.loc[(v.iloc[:,1]==True),'index'])
u = (train[col_filtered].nunique()<11).reset_index()
col_filtered = list(u.loc[(u.iloc[:,1]==True),'index'])

### Target mean encoding on col_filtered 

In [100]:
def mean_encoding(df, test, col):
    for item in col:
        mean_device_type = df.groupby(item).is_female.mean()
        test[item+"_mean_enc"] = test[item].map(mean_device_type)
        df[item+"_mean_enc"] = df[item].map(mean_device_type)
    return df,test

In [101]:
def process_data(train,test):
    combine = pd.concat([train,test])
    object_cols = list(test.select_dtypes(include=['object','category']).columns)
    for c in object_cols:
        combine[c] = combine[c].astype('category').cat.codes+1  
    train = combine.iloc[:train.shape[0],]
    test = combine.iloc[train.shape[0]:,]
    return train,test

In [102]:
train,test = mean_encoding(train, test, col_filtered)
train,test = process_data(train,test)

In [103]:
global_mean = train.is_female.mean()
test.fillna(global_mean, inplace=True)

In [104]:
del test['is_female']
test.head(3), train.shape

(   AA14  AA15  AA3  AA3_mean_enc  AA4   AA5  AA5_mean_enc   AA6  AA6_mean_enc  \
 0  4479   535    4      0.541462   41  1000      0.538803     7      0.534333   
 1  3803   476    3      0.476144   32     2      0.524664  1000      0.533345   
 2  5610   585    3      0.476144   36     5      0.528686  1000      0.533345   
 
       AA7    ...      MT6B  MT6B_mean_enc  MT6C  MT6_mean_enc   MT7  MT7A  \
 0  417211    ...      1000       0.701275  1000      0.701275     2  1000   
 1  322011    ...         1       0.363453     0      0.179292  1000  1000   
 2  365011    ...      1000       0.701275  1000      0.701275     2  1000   
 
    MT7A_mean_enc   MT8   MT9  row_nulls  
 0       0.488904  1000    12          0  
 1       0.488904  1000  1000          0  
 2       0.488904  1000    13          0  
 
 [3 rows x 888 columns], (18255, 889))

In [105]:
# mn.bar(test)

In [106]:
# train_filled_knn = KNN(k=3).complete(train)
# train_filled_knn = pd.DataFrame(train_filled_knn, columns=train.columns, index = train.index)
# train_filled_knn.shape

In [107]:
# test_filled_knn = KNN(k=3).complete(test)
# test_filled_knn = pd.DataFrame(test_filled_knn, columns=test.columns, index = test.index)
# test_filled_knn.shape

In [2]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [3]:
train = pd.read_csv("train_afterbin.csv")
test = pd.read_csv("test_afterbin.csv")

In [4]:
y = train['is_female']
X = train.drop(['is_female'], axis = 1)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=10)

In [110]:
def auc(m, train, val): 
    return (metrics.roc_auc_score(y_train,m.predict_proba(train)[:,1]),
                            metrics.roc_auc_score(y_val,m.predict_proba(val)[:,1]))

### Parameter Tuning Random Search XGBoost

In [112]:
# model = xgb.XGBClassifier()
# param_dist = {"max_depth": [15, 20, 25,30],
#               "min_child_weight" : [3,5,10]}

# # run randomized search
# n_iter_search = 10
# random_search = RandomizedSearchCV(model, param_distributions=param_dist, cv = 4, 
#                                    n_iter=n_iter_search,verbose=1, n_jobs=-1)
# random_search.fit(X_train, y_train)
# random_search.best_estimator_

### Fitting Final Model XGBoost

In [ ]:
#model = xgb.XGBClassifier(max_depth=20, min_child_weight=8,  n_estimators=100, n_jobs=-1 )



#model.fit(X_train,y_train)

#auc(model, X_train, X_val)

In [5]:
#model = xgb.XGBClassifier(max_depth=20, min_child_weight=8,  n_estimators=100)
model = xgb.XGBClassifier(max_depth=6, min_child_weight=1.36,  n_estimators=540, colsample_bytree= 0.6, gamma= 0.05, subsample=1, learning_rate=0.05)
#model.fit(X_train,y_train)
#colsample_bytree
#Depth: 6 Eta: 0.05 Subsample: 1 Colsample: 0.6 Gamma: 0.05 Minchild weight: 1.36 Alpha: 0"
#auc(model, X_train, X_val)

#(0.9990936995241299, 0.9711375605506248)
#model = xgb.XGBClassifier(max_depth=6, min_child_weight=1.36,  n_estimators=540, colsample_bytree= 0.6, gamma= 0.05, subsample=1, learning_rate=0.05)

In [6]:
model.fit(X,y)
pred = model.predict_proba(test)

In [7]:
submit = pd.DataFrame()
submit['test_id'] = id
submit['is_female'] = pred[:,1]
submit.to_csv("Submission_xgboost.csv", index=False)

In [114]:
# test.loc[:,'is_female'] = pred[:,1]
# test.loc[(test['DL2_96_OTHERS']=="House wife"),'is_female']
# test['DL2_96_OTHERS']=="House wife"
# (train[col_filtered].nunique()>10).reset_index()

### Catboost Classifier - Base model

In [64]:
cate_features_index = np.where(X.dtypes != float)[0]

In [116]:
# train default catBoost classifier. Default loss metric is LogLoss (lower is better)
clf = CatBoostClassifier(eval_metric="AUC",one_hot_max_size=6, iterations=400)

In [ ]:
# clf.fit(X,Y , cat_features=cat_dims)
clf.fit(X_train,y_train, cat_features=cate_features_index)

In [86]:
auc(clf, X_train, X_val)

(0.9753617349262516, 0.9690407385145595)

In [ ]:
clf.fit(X,y, cat_features=cate_features_index)

In [118]:
pred = clf.predict_proba(test)

In [119]:
submit = pd.DataFrame()
submit['test_id'] = id
submit['is_female'] = pred[:,1]
submit.to_csv("Submission_base_Cat.csv", index=False)

### Parameter Tuning

In [164]:
from sortedcontainers import SortedList
import copy
import collections
import numpy as np
from itertools import product,chain
import pandas
from sklearn.model_selection import KFold
import catboost as cb

''' a class for doing grid search on a set of parameters provided in a dict. 'pdict' should be a dictionary like the following:
pdict = {'depth':[1,2], 'iterations':[250,100,500], 'thread_count':4}

when grid_search is called it will return an iterator that provides samples from the dictionary e.g.
{'depth':1, 'iterations':250, 'thread_count':4}
{'depth':2, 'iterations':250, 'thread_count':4}
{'depth':1, 'iterations':100, 'thread_count':4}
etc.
after calling an iteration of grid_search, you need to test the classifier and run 'register_result'
This will update the internal list of results, so that the next call to grid_search will use the best
parameters for all the parameters not currently being updated.

grid_search can be provided a list e.g. grid_search(['depth']) this will use the current best parameters for all
the other arguments and only search over 'depth'. You can then call e.g. grid_search(['iterations']) and it will use
the best depth found previously and cycle through all the 'iterations'. Searching incrementally can be much faster
than doing a full grid search, but may miss the global optimum. '''
class paramsearch:
    def __init__(self,pdict):    
        self.pdict = {}
        # if something is not passed in as a sequence, make it a sequence with 1 element
        #   don't treat strings as sequences
        for a,b in pdict.items():
            if isinstance(b, collections.Sequence) and not isinstance(b, str): self.pdict[a] = b
            else: self.pdict[a] = [b]
        # our results are a sorted list, so the best score is always the final element
        self.results = SortedList()       
                    
    def grid_search(self,keys=None):
        # do grid search on only the keys listed. If none provided, do all
        if keys==None: keylist = self.pdict.keys()
        else: keylist = keys
 
        listoflists = [] # this will be list of lists of key,value pairs
        for key in keylist: listoflists.append([(key,i) for i in self.pdict[key]])
        for p in product(*listoflists):
            # do any changes to the current best parameter set
            if len(self.results)>0: template = self.results[-1][1]
            else: template = {a:b[0] for a,b in self.pdict.items()}
            # if our updates are the same as current best, don't bother
            if self.equaldict(dict(p),template): continue
            # take the current best and update just the ones to change
            yield self.overwritedict(dict(p),template)
                              
    def equaldict(self,a,b):
        for key in a.keys(): 
            if a[key] != b[key]: return False
        return True            
                              
    def overwritedict(self,new,old):
        old = copy.deepcopy(old)
        for key in new.keys(): old[key] = new[key]
        return old            
    
    # save a (score,params) pair to results. Since 'results' is a sorted list,
    #   the best score is always the final element. A small amount of noise is added
    #   because sorted lists don't like it when two scores are exactly the same    
    def register_result(self,result,params):
        self.results.add((result+np.random.randn()*1e-10,params))    
        
    def bestscore(self):
        return self.results[-1][0]
        
    def bestparam(self):
        return self.results[-1][1]

In [165]:
params = {'depth':[2,5,10],
          'iterations':[100],
          'learning_rate':[0.03,0.6,0.3], 
          'l2_leaf_reg':[1,5,30]}

In [166]:
# this function does 3-fold crossvalidation with catboostclassifier          
def crossvaltest(params,train_set,train_label,cat_dims,n_splits=3):
    kf = KFold(n_splits=n_splits,shuffle=True) 
    res = []
    for train_index, test_index in kf.split(train_set):
        train = train_set.iloc[train_index,:]
        test = train_set.iloc[test_index,:]

        labels = train_label.ix[train_index]
        test_labels = train_label.ix[test_index]

        clf = cb.CatBoostClassifier(**params)
        clf.fit(train, np.ravel(labels), cat_features=cat_dims)

        res.append(np.mean(clf.predict(test)==np.ravel(test_labels)))
    return np.mean(res)

In [168]:
# this function runs grid search on several parameters
def catboost_param_tune(params,train_set,train_label,cat_dims=None,n_splits=3):
    ps = paramsearch(params)
    # search 'border_count', 'l2_leaf_reg' etc. individually 
    #   but 'iterations','learning_rate' together
    for prms in chain(ps.grid_search(['l2_leaf_reg']),
                      ps.grid_search(['iterations','learning_rate']),
                      ps.grid_search(['depth'])):
        res = crossvaltest(prms,train_set,train_label,cat_dims,n_splits)
        # save the crossvalidation result so that future iterations can reuse the best parameters
        ps.register_result(res,prms)
        print(res,prms,'best:',ps.bestscore(),ps.bestparam())
    return ps.bestparam()

bestparams = catboost_param_tune(params,X_train,y_train,cat_dims =cate_features_index)

0:	learn: 0.6925585	total: 575ms	remaining: 56.9s
1:	learn: 0.6920207	total: 1.03s	remaining: 50.4s
2:	learn: 0.6915279	total: 1.6s	remaining: 51.6s
3:	learn: 0.6910446	total: 2.06s	remaining: 49.5s
4:	learn: 0.6906114	total: 2.19s	remaining: 41.7s
5:	learn: 0.6901705	total: 2.57s	remaining: 40.3s
6:	learn: 0.6897793	total: 2.98s	remaining: 39.5s
7:	learn: 0.6894184	total: 3.35s	remaining: 38.5s
8:	learn: 0.6890683	total: 3.73s	remaining: 37.8s
9:	learn: 0.6887096	total: 4.2s	remaining: 37.8s
10:	learn: 0.6883877	total: 4.58s	remaining: 37s
11:	learn: 0.6881054	total: 4.69s	remaining: 34.4s
12:	learn: 0.6878181	total: 5.05s	remaining: 33.8s
13:	learn: 0.6875684	total: 5.17s	remaining: 31.8s
14:	learn: 0.6872968	total: 5.53s	remaining: 31.3s
15:	learn: 0.6870631	total: 5.86s	remaining: 30.8s
16:	learn: 0.6868480	total: 6.2s	remaining: 30.3s
17:	learn: 0.6866455	total: 6.65s	remaining: 30.3s
18:	learn: 0.6864616	total: 6.78s	remaining: 28.9s
19:	learn: 0.6862831	total: 7.15s	remaining: 2

62:	learn: 0.6811000	total: 22.4s	remaining: 13.2s
63:	learn: 0.6810771	total: 22.8s	remaining: 12.8s
64:	learn: 0.6810618	total: 23.1s	remaining: 12.5s
65:	learn: 0.6810393	total: 23.5s	remaining: 12.1s
66:	learn: 0.6810237	total: 23.9s	remaining: 11.8s
67:	learn: 0.6809976	total: 24.2s	remaining: 11.4s
68:	learn: 0.6809867	total: 24.3s	remaining: 10.9s
69:	learn: 0.6809644	total: 24.7s	remaining: 10.6s
70:	learn: 0.6809083	total: 25s	remaining: 10.2s
71:	learn: 0.6808642	total: 25.3s	remaining: 9.84s
72:	learn: 0.6808178	total: 25.6s	remaining: 9.48s
73:	learn: 0.6807919	total: 26s	remaining: 9.12s
74:	learn: 0.6807690	total: 26.3s	remaining: 8.77s
75:	learn: 0.6807423	total: 26.6s	remaining: 8.41s
76:	learn: 0.6807054	total: 26.9s	remaining: 8.05s
77:	learn: 0.6806912	total: 27.3s	remaining: 7.69s
78:	learn: 0.6806825	total: 27.6s	remaining: 7.34s
79:	learn: 0.6806490	total: 27.9s	remaining: 6.98s
80:	learn: 0.6806201	total: 28.3s	remaining: 6.63s
81:	learn: 0.6805811	total: 28.6s	r

21:	learn: 0.6857807	total: 6.55s	remaining: 23.2s
22:	learn: 0.6856229	total: 6.9s	remaining: 23.1s
23:	learn: 0.6854799	total: 7.02s	remaining: 22.2s
24:	learn: 0.6853384	total: 7.34s	remaining: 22s
25:	learn: 0.6852091	total: 7.66s	remaining: 21.8s
26:	learn: 0.6850762	total: 8s	remaining: 21.6s
27:	learn: 0.6849571	total: 8.31s	remaining: 21.4s
28:	learn: 0.6848476	total: 8.66s	remaining: 21.2s
29:	learn: 0.6847256	total: 9.09s	remaining: 21.2s
30:	learn: 0.6846036	total: 9.65s	remaining: 21.5s
31:	learn: 0.6845158	total: 9.86s	remaining: 21s
32:	learn: 0.6844229	total: 10.2s	remaining: 20.7s
33:	learn: 0.6843440	total: 10.7s	remaining: 20.8s
34:	learn: 0.6842709	total: 10.8s	remaining: 20.1s
35:	learn: 0.6841835	total: 11.2s	remaining: 19.9s
36:	learn: 0.6841070	total: 11.6s	remaining: 19.7s
37:	learn: 0.6840462	total: 11.7s	remaining: 19.1s
38:	learn: 0.6839890	total: 11.8s	remaining: 18.5s
39:	learn: 0.6839339	total: 12.2s	remaining: 18.3s
40:	learn: 0.6838821	total: 12.5s	remai

84:	learn: 0.6812313	total: 32.9s	remaining: 5.81s
85:	learn: 0.6812027	total: 33.4s	remaining: 5.43s
86:	learn: 0.6811641	total: 33.8s	remaining: 5.05s
87:	learn: 0.6811406	total: 34.2s	remaining: 4.66s
88:	learn: 0.6811086	total: 34.6s	remaining: 4.28s
89:	learn: 0.6810774	total: 35s	remaining: 3.89s
90:	learn: 0.6810373	total: 35.4s	remaining: 3.5s
91:	learn: 0.6809946	total: 35.8s	remaining: 3.11s
92:	learn: 0.6809622	total: 36.2s	remaining: 2.73s
93:	learn: 0.6809567	total: 36.7s	remaining: 2.35s
94:	learn: 0.6809452	total: 37.2s	remaining: 1.96s
95:	learn: 0.6809280	total: 37.6s	remaining: 1.57s
96:	learn: 0.6809209	total: 38.1s	remaining: 1.18s
97:	learn: 0.6809076	total: 38.5s	remaining: 785ms
98:	learn: 0.6808685	total: 38.9s	remaining: 393ms
99:	learn: 0.6808492	total: 39.3s	remaining: 0us
0:	learn: 0.6926172	total: 482ms	remaining: 47.8s
1:	learn: 0.6921250	total: 896ms	remaining: 43.9s
2:	learn: 0.6916613	total: 1.45s	remaining: 46.9s
3:	learn: 0.6912243	total: 1.94s	remain

43:	learn: 0.6730714	total: 23.6s	remaining: 30s
44:	learn: 0.6725586	total: 23.9s	remaining: 29.2s
45:	learn: 0.6725227	total: 24.2s	remaining: 28.5s
46:	learn: 0.6724820	total: 24.6s	remaining: 27.7s
47:	learn: 0.6722739	total: 24.9s	remaining: 27s
48:	learn: 0.6721693	total: 25.3s	remaining: 26.3s
49:	learn: 0.6720214	total: 25.7s	remaining: 25.7s
50:	learn: 0.6714248	total: 26s	remaining: 25s
51:	learn: 0.6712611	total: 26.4s	remaining: 24.4s
52:	learn: 0.6708426	total: 26.8s	remaining: 23.7s
53:	learn: 0.6707613	total: 27.1s	remaining: 23.1s
54:	learn: 0.6704678	total: 27.5s	remaining: 22.5s
55:	learn: 0.6702314	total: 27.8s	remaining: 21.9s
56:	learn: 0.6699980	total: 28.2s	remaining: 21.3s
57:	learn: 0.6699636	total: 28.5s	remaining: 20.7s
58:	learn: 0.6699205	total: 28.9s	remaining: 20.1s
59:	learn: 0.6696523	total: 29.2s	remaining: 19.5s
60:	learn: 0.6696077	total: 29.6s	remaining: 18.9s
61:	learn: 0.6695811	total: 29.9s	remaining: 18.3s
62:	learn: 0.6692241	total: 30.3s	remai

...

28:	learn: 0.6814762	total: 11.3s	remaining: 27.6s
29:	learn: 0.6812588	total: 11.7s	remaining: 27.3s
30:	learn: 0.6811724	total: 12.1s	remaining: 26.9s
31:	learn: 0.6810758	total: 12.5s	remaining: 26.5s
32:	learn: 0.6810541	total: 12.9s	remaining: 26.1s
33:	learn: 0.6807913	total: 13.3s	remaining: 25.8s
34:	learn: 0.6806560	total: 13.7s	remaining: 25.4s
35:	learn: 0.6806299	total: 14.1s	remaining: 25s
36:	learn: 0.6804264	total: 14.5s	remaining: 24.6s
37:	learn: 0.6803615	total: 14.8s	remaining: 24.2s
38:	learn: 0.6803032	total: 15.2s	remaining: 23.8s
39:	learn: 0.6802783	total: 15.7s	remaining: 23.5s
40:	learn: 0.6799835	total: 16.1s	remaining: 23.1s
41:	learn: 0.6798348	total: 16.5s	remaining: 22.7s
42:	learn: 0.6798269	total: 16.9s	remaining: 22.4s
43:	learn: 0.6797620	total: 17.3s	remaining: 22s
44:	learn: 0.6795959	total: 17.6s	remaining: 21.5s
45:	learn: 0.6794495	total: 18s	remaining: 21.2s
46:	learn: 0.6792821	total: 18.4s	remaining: 20.8s
47:	learn: 0.6790610	total: 18.8s	rem

85:	learn: 0.6236975	total: 2m 22s	remaining: 23.2s
86:	learn: 0.6229876	total: 2m 23s	remaining: 21.5s
87:	learn: 0.6229341	total: 2m 26s	remaining: 20s
88:	learn: 0.6215555	total: 2m 28s	remaining: 18.3s
89:	learn: 0.6204515	total: 2m 30s	remaining: 16.7s
90:	learn: 0.6194023	total: 2m 31s	remaining: 15s
91:	learn: 0.6192697	total: 2m 33s	remaining: 13.4s
92:	learn: 0.6187098	total: 2m 35s	remaining: 11.7s
93:	learn: 0.6182047	total: 2m 37s	remaining: 10s
94:	learn: 0.6177026	total: 2m 39s	remaining: 8.38s
95:	learn: 0.6174145	total: 2m 40s	remaining: 6.7s
96:	learn: 0.6169440	total: 2m 42s	remaining: 5.03s
97:	learn: 0.6168842	total: 2m 44s	remaining: 3.35s
98:	learn: 0.6146525	total: 2m 45s	remaining: 1.67s
99:	learn: 0.6143123	total: 2m 47s	remaining: 0us
0:	learn: 0.6837651	total: 1.89s	remaining: 3m 7s
1:	learn: 0.6825000	total: 2.78s	remaining: 2m 16s
2:	learn: 0.6818080	total: 4.66s	remaining: 2m 30s
3:	learn: 0.6804729	total: 6.45s	remaining: 2m 34s
4:	learn: 0.6799047	total:

44:	learn: 0.6472647	total: 1m 12s	remaining: 1m 28s
45:	learn: 0.6463593	total: 1m 14s	remaining: 1m 27s
46:	learn: 0.6460204	total: 1m 15s	remaining: 1m 25s
47:	learn: 0.6458226	total: 1m 17s	remaining: 1m 24s
48:	learn: 0.6455119	total: 1m 19s	remaining: 1m 22s
49:	learn: 0.6446101	total: 1m 21s	remaining: 1m 21s
50:	learn: 0.6435407	total: 1m 22s	remaining: 1m 19s
51:	learn: 0.6430322	total: 1m 24s	remaining: 1m 17s
52:	learn: 0.6421875	total: 1m 26s	remaining: 1m 16s
53:	learn: 0.6415823	total: 1m 28s	remaining: 1m 15s
54:	learn: 0.6412868	total: 1m 31s	remaining: 1m 14s
55:	learn: 0.6402522	total: 1m 33s	remaining: 1m 13s
56:	learn: 0.6397129	total: 1m 36s	remaining: 1m 13s
57:	learn: 0.6392447	total: 1m 38s	remaining: 1m 11s
58:	learn: 0.6385574	total: 1m 40s	remaining: 1m 10s
59:	learn: 0.6368813	total: 1m 43s	remaining: 1m 8s
60:	learn: 0.6366626	total: 1m 44s	remaining: 1m 6s
61:	learn: 0.6347427	total: 1m 46s	remaining: 1m 5s
62:	learn: 0.6336456	total: 1m 48s	remaining: 1m 

96:	learn: 0.3563082	total: 17m 22s	remaining: 32.2s
97:	learn: 0.3541592	total: 17m 34s	remaining: 21.5s
98:	learn: 0.3505120	total: 17m 45s	remaining: 10.8s
99:	learn: 0.3475971	total: 17m 56s	remaining: 0us
0:	learn: 0.6845282	total: 625ms	remaining: 1m 1s
1:	learn: 0.6805389	total: 13.7s	remaining: 11m 12s
2:	learn: 0.6796342	total: 16.1s	remaining: 8m 40s
3:	learn: 0.6786098	total: 18.4s	remaining: 7m 20s
4:	learn: 0.6698149	total: 29.6s	remaining: 9m 22s
5:	learn: 0.6652497	total: 42s	remaining: 10m 57s
6:	learn: 0.6643685	total: 43.8s	remaining: 9m 41s
7:	learn: 0.6643016	total: 44.5s	remaining: 8m 31s
8:	learn: 0.6641897	total: 44.9s	remaining: 7m 34s
9:	learn: 0.6618783	total: 55.8s	remaining: 8m 22s
10:	learn: 0.6606309	total: 1m 7s	remaining: 9m 8s
11:	learn: 0.6596712	total: 1m 10s	remaining: 8m 36s
12:	learn: 0.6563684	total: 1m 22s	remaining: 9m 13s
13:	learn: 0.6530707	total: 1m 33s	remaining: 9m 34s
14:	learn: 0.6493400	total: 1m 47s	remaining: 10m 10s
15:	learn: 0.6433

51:	learn: 0.4863944	total: 9m 22s	remaining: 8m 38s
52:	learn: 0.4791312	total: 9m 31s	remaining: 8m 26s
53:	learn: 0.4772113	total: 9m 42s	remaining: 8m 16s
54:	learn: 0.4691514	total: 9m 53s	remaining: 8m 5s
55:	learn: 0.4635708	total: 10m 6s	remaining: 7m 56s
56:	learn: 0.4564093	total: 10m 18s	remaining: 7m 46s
57:	learn: 0.4507161	total: 10m 28s	remaining: 7m 35s
58:	learn: 0.4452220	total: 10m 46s	remaining: 7m 29s
59:	learn: 0.4422515	total: 10m 54s	remaining: 7m 16s
60:	learn: 0.4377459	total: 11m 5s	remaining: 7m 5s
61:	learn: 0.4337453	total: 11m 16s	remaining: 6m 54s
62:	learn: 0.4327770	total: 11m 27s	remaining: 6m 44s
63:	learn: 0.4302784	total: 11m 37s	remaining: 6m 32s
64:	learn: 0.4270416	total: 11m 50s	remaining: 6m 22s
65:	learn: 0.4240554	total: 12m 4s	remaining: 6m 13s
66:	learn: 0.4202616	total: 12m 13s	remaining: 6m 1s
67:	learn: 0.4182826	total: 12m 25s	remaining: 5m 50s
68:	learn: 0.4117529	total: 12m 36s	remaining: 5m 39s
69:	learn: 0.4111234	total: 12m 48s	re

In [169]:
bestparams

{'depth': 10, 'iterations': 100, 'l2_leaf_reg': 5, 'learning_rate': 0.6}

In [174]:
clf = CatBoostClassifier(eval_metric="AUC",one_hot_max_size=11, \
                        depth=10, iterations= 300, l2_leaf_reg= 5, learning_rate= 0.08)
clf.fit(X_train,y_train, cat_features=cate_features_index)
auc(clf, X_train, X_val)

0:	learn: 0.9460698	total: 1.01s	remaining: 5m 2s
1:	learn: 0.9508904	total: 2.14s	remaining: 5m 19s
2:	learn: 0.9539938	total: 3.27s	remaining: 5m 24s
3:	learn: 0.9545475	total: 4.53s	remaining: 5m 35s
4:	learn: 0.9541837	total: 5.07s	remaining: 4m 58s
5:	learn: 0.9563207	total: 6.31s	remaining: 5m 9s
6:	learn: 0.9568048	total: 7.32s	remaining: 5m 6s
7:	learn: 0.9581269	total: 8.37s	remaining: 5m 5s
8:	learn: 0.9590676	total: 9.29s	remaining: 5m
9:	learn: 0.9602286	total: 10.5s	remaining: 5m 5s
10:	learn: 0.9615204	total: 11.4s	remaining: 4m 59s
11:	learn: 0.9618896	total: 12.5s	remaining: 5m
12:	learn: 0.9626751	total: 13.9s	remaining: 5m 7s
13:	learn: 0.9630140	total: 15.2s	remaining: 5m 10s
14:	learn: 0.9635642	total: 16.5s	remaining: 5m 12s
15:	learn: 0.9636841	total: 17.3s	remaining: 5m 6s
16:	learn: 0.9638885	total: 18.2s	remaining: 5m 2s
17:	learn: 0.9641976	total: 19.5s	remaining: 5m 4s
18:	learn: 0.9645208	total: 20s	remaining: 4m 55s
19:	learn: 0.9648995	total: 20.8s	remaini

157:	learn: 0.9848531	total: 3m 59s	remaining: 3m 35s
158:	learn: 0.9849035	total: 4m 1s	remaining: 3m 33s
159:	learn: 0.9850035	total: 4m 2s	remaining: 3m 32s
160:	learn: 0.9850742	total: 4m 4s	remaining: 3m 30s
161:	learn: 0.9851463	total: 4m 5s	remaining: 3m 29s
162:	learn: 0.9851734	total: 4m 7s	remaining: 3m 27s
163:	learn: 0.9853167	total: 4m 8s	remaining: 3m 26s
164:	learn: 0.9853709	total: 4m 9s	remaining: 3m 24s
165:	learn: 0.9854157	total: 4m 11s	remaining: 3m 22s
166:	learn: 0.9855111	total: 4m 12s	remaining: 3m 21s
167:	learn: 0.9855906	total: 4m 14s	remaining: 3m 19s
168:	learn: 0.9856373	total: 4m 16s	remaining: 3m 19s
169:	learn: 0.9856529	total: 4m 19s	remaining: 3m 18s
170:	learn: 0.9857158	total: 4m 21s	remaining: 3m 17s
171:	learn: 0.9857627	total: 4m 23s	remaining: 3m 15s
172:	learn: 0.9858524	total: 4m 25s	remaining: 3m 14s
173:	learn: 0.9858871	total: 4m 26s	remaining: 3m 13s
174:	learn: 0.9859929	total: 4m 28s	remaining: 3m 11s
175:	learn: 0.9861041	total: 4m 29s

(0.9892740475048535, 0.9716071644528197)

In [175]:
clf.fit(X,y, cat_features=cate_features_index)
pred = clf.predict_proba(test)

0:	learn: 0.9471319	total: 1.42s	remaining: 7m 5s
1:	learn: 0.9508789	total: 2.83s	remaining: 7m 2s
2:	learn: 0.9545801	total: 4.18s	remaining: 6m 53s
3:	learn: 0.9554333	total: 5.54s	remaining: 6m 49s
4:	learn: 0.9560652	total: 6.93s	remaining: 6m 48s
5:	learn: 0.9573263	total: 8.68s	remaining: 7m 5s
6:	learn: 0.9575294	total: 10.4s	remaining: 7m 13s
7:	learn: 0.9583815	total: 11.7s	remaining: 7m 7s
8:	learn: 0.9596003	total: 13.2s	remaining: 7m 5s
9:	learn: 0.9606575	total: 14.5s	remaining: 6m 59s
10:	learn: 0.9609851	total: 16s	remaining: 7m 1s
11:	learn: 0.9616226	total: 17.5s	remaining: 6m 59s
12:	learn: 0.9617530	total: 18.2s	remaining: 6m 41s
13:	learn: 0.9625939	total: 19.4s	remaining: 6m 36s
14:	learn: 0.9631376	total: 21s	remaining: 6m 38s
15:	learn: 0.9634541	total: 22.1s	remaining: 6m 31s
16:	learn: 0.9639755	total: 23.7s	remaining: 6m 34s
17:	learn: 0.9645184	total: 25.7s	remaining: 6m 43s
18:	learn: 0.9648833	total: 27.8s	remaining: 6m 50s
19:	learn: 0.9650637	total: 29.3

157:	learn: 0.9839393	total: 4m 44s	remaining: 4m 15s
158:	learn: 0.9840684	total: 4m 46s	remaining: 4m 14s
159:	learn: 0.9841564	total: 4m 48s	remaining: 4m 12s
160:	learn: 0.9842089	total: 4m 50s	remaining: 4m 11s
161:	learn: 0.9842709	total: 4m 53s	remaining: 4m 9s
162:	learn: 0.9843235	total: 4m 55s	remaining: 4m 8s
163:	learn: 0.9845133	total: 4m 57s	remaining: 4m 6s
164:	learn: 0.9846070	total: 5m	remaining: 4m 5s
165:	learn: 0.9847001	total: 5m 2s	remaining: 4m 4s
166:	learn: 0.9849647	total: 5m 4s	remaining: 4m 2s
167:	learn: 0.9850972	total: 5m 6s	remaining: 4m
168:	learn: 0.9851784	total: 5m 8s	remaining: 3m 59s
169:	learn: 0.9852167	total: 5m 10s	remaining: 3m 57s
170:	learn: 0.9853158	total: 5m 12s	remaining: 3m 56s
171:	learn: 0.9854028	total: 5m 14s	remaining: 3m 54s
172:	learn: 0.9854368	total: 5m 17s	remaining: 3m 53s
173:	learn: 0.9854970	total: 5m 20s	remaining: 3m 51s
174:	learn: 0.9855661	total: 5m 22s	remaining: 3m 50s
175:	learn: 0.9856477	total: 5m 24s	remaining:

In [176]:
submit = pd.DataFrame()
submit['test_id'] = id
submit['is_female'] = pred[:,1]
submit.to_csv("Submission_tuned_Cat.csv", index=False)

In [177]:
pred_mean = (pred[:,1] + pd.read_csv("Submission_xgboost.csv")['is_female'])/2

In [183]:
submit['is_female'] = pred_mean
submit.to_csv("FirstEnsemble.csv", index=False)

In [184]:
pred_mean3 = (pred[:,1] + pd.read_csv("Submission_xgboost.csv")['is_female'] +\
             pd.read_csv("Submission_base_Cat.csv")['is_female'])/3
submit['is_female'] = pred_mean3
submit.to_csv("SecondEnsemble.csv", index=False)

### Random Forest

In [55]:
m = RandomForestClassifier(n_estimators=200, max_features= 100,min_samples_leaf=8, max_depth=80)
m.fit(X_train, y_train)
auc(m, X_train, X_val)

(0.9899227020756343, 0.9648185150889961)

In [ ]:
m.fit(X,y)
pred = m.predict_proba(test)

In [ ]:
submit = pd.DataFrame()
submit['test_id'] = id
submit['is_female'] = pred[:,1]
submit.to_csv("Submission_RF.csv", index=False)

### Logistic Regression after Normalizing

In [21]:
# scaler = StandardScaler()
# scaler.fit(train_filled_knn.drop(['is_female'], axis =1))
# scaled_train = scaler.transform(train_filled_knn.drop(['is_female'], axis =1))
# scaled_test = scaler.transform(test_filled_knn)

# scaled_train = pd.DataFrame(scaled_train, columns=train_filled_knn.drop(['is_female'], axis =1).columns, index=train_filled_knn.drop(['is_female'], axis =1).index)
# scaled_test = pd.DataFrame(scaled_test, columns=test_filled_knn.columns, index=test_filled_knn.index)

# scaled_train["is_female"] =  train_filled_knn["is_female"]
# x_train ,x_val = train_test_split(scaled_train,test_size=0.2) 

# y_train = x_train["is_female"]
# del x_train["is_female"]
# y_valid = x_val['is_female']
# del x_val['is_female']

# def auclr(lr, train, val): 
#     return (metrics.roc_auc_score(y_train,lr.predict_proba(train)[:,1]),
#                             metrics.roc_auc_score(y_valid,lr.predict_proba(val)[:,1]))
# lr = LogisticRegression()
# lr.fit(x_train, y_train)
# auclr(lr, x_train, x_val)

# y_train = scaled_train["is_female"]
# x_train = scaled_train.copy()
# del x_train["is_female"]
# lr.fit(x_train, y_train)

# submit = pd.DataFrame()
# submit['test_id'] = id
# submit['is_female'] = lr.predict_proba(scaled_test)[:,1]
# submit.to_csv("Submission_LR.csv", index=False)